# **Tools and Agents**

## **Imports**

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool
from langchain import hub
from platform import python_version
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.runnables import chain
from langchain.memory import ConversationSummaryMemory
from operator import itemgetter


## **Reading Files**

In [ ]:

##-----------Reading a PDf File------------------
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()
docs



#--------------Removing the newlines (\n) ----------------
for i in docs:
    i.page_content = ' '.join(i.page_content.split())
#docs


## **Document Splitting**

In [ ]:

###Use RecursiveCharacterTextSplitter:For structured or hierarchical text where preserving logical units is important.When working with content like articles, legal documents, or code.
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50) # Here chunk_overlap means last 50 words will be repeated in the next chunk
pages_char_split=text_splitter.split_documents(docs)  # This will return the list of documents,in the documents formate




' #For a perfect View\nfor page in pages_char_split:\n    d = page.page_content  \n    print(d) '

## **Embeddings**

In [ ]:


##-----------Hugging Face -------------------------
import os
from dotenv import load_dotenv
HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN is None:print("API key not found. Ensure it is set in the .env file.")
else: print("API key loaded successfully.")

#add langchain_huggingface to requirements.txt
from langchain_huggingface import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

API key loaded successfully.


f:\Generative_Ai\end_to_end_projects\APP _first\Master\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **VectorStores**

In [ ]:


##------------ChromaDB--------------------
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=docs,embedding=embedding)


'  vectorstore_from_directory.delete("55409552-1943-4892-949a-3b475ff9c840")'

## **Retriever**

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr',  # এখানে আমরা ইচ্ছা মতো সার্চ টাইপ গুলো ঠিক করতে পারব
    search_kwargs={
        'k': 3,  # Number of top results to retrieve
        'lambda_mult': 0.7  # Diversity parameter (used for 'mmr')
    }
)

# Other possible search_type values:
# 'mmr' (Maximal Marginal Relevance): Balances relevance and diversity in results
# 'similarity': Retrieves documents based on similarity to the query.
# 'ranked': Ranks documents by relevance scores (e.g., cosine similarity).
# 'similarity_score_threshold': Retrieves documents exceeding a minimum similarity score.
# 'approximate': Uses approximate nearest neighbors for faster but less precise results.
# 'hybrid': Combines vector-based similarity with other search strategies (e.g., keyword search).
# 'lexical': Traditional keyword-based search (e.g., BM25 or TF-IDF).
# 'weighted': Adjusts retrieval by assigning weights to certain terms or document sections.


#question = ""
#retrieved_docs = retriever.invoke(question) '''



## **Model Creation**

In [ ]:
# -------------------- Model Setup ------------------

from langchain_groq import ChatGroq
load_dotenv()  
groq_api_key = os.getenv("GROQ_API_KEY")
chat = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
#chat=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

## **Tools creation**
**Tools List: https://python.langchain.com/docs/integrations/tools/**

In [ ]:
##---------------------------------------Converting Retriever to a Tool---------------------------------------
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Mathematical_Cosmology", 
                                       description = '''For any questions regarding Mathematical Cosmology, you must use this tool.''')

#retriever_tool.name #To see the Name of the tool
#retriever_tool.args #To see the Arguments of the tool
#retriever_tool.description #To see the Description of the tool





##---------------------------------------Converting the LLM to a tool---------------------------------
'''from langchain.tools import Tool

def chat_with_model(input_text: str) -> str:
    return chat.invoke(input_text)

chat_tool = Tool(
    name="ChatGroq",
    func=chat_with_model,  # Use the function defined above
    description="For any normal conversation, you must use this tool."
)'''

#chat_tool.name #To see the Name of the tool
#chat_tool.args #To see the Arguments of the tool
#chat_tool.description #To see the Description of the tool








##---------------------------------------Arxiv--Research---------------------------------------
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

#arxiv.name  #To see the Name of the tool
#arxiv.args  #To see the Arguments of the tool
#arxiv.description  #To see the Description of the tool



##---------------------------------------Wikipedia_Research---------------------------------------
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

#wiki.name #To see the Name of the tool
#wiki.args #To see the Arguments of the tool
#wiki.description #To see the Description of the tool











#------------------Combining the tools------------------
tools = [wiki, arxiv, retriever_tool]
#tools

## **Prompt Template**

**Here we will use the inbuild chat prompt template**

**Details link :https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_functions_agent/**

In [ ]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## **Agents**
**Types of agents:https://python.langchain.com/v0.1/docs/modules/agents/agent_types/**

In [ ]:
# Agent 01: 
# This agent is designed to interact with specific tools by calling them when needed.It can perform tasks by leveraging the capabilities of the provided 
# tools and the prompt structure. Use this agent when you want the agent to execute a task using defined tools without OpenAI-specific tools integration.

from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm = chat, 
                                  tools = tools, 
                                  prompt = prompt)




# Agent 02:
# This agent is tailored for working with OpenAI's built-in tools and APIs.It is useful when you want the agent to leverage OpenAI tools 
# (e.g., embeddings, file operations, or OpenAI plugins) alongside custom tools.Use this agent when integrating OpenAI tools with your workflow is necessary.

'''
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm = chat, 
                                  tools = tools, 
                                  prompt = prompt)
'''

agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

## **Agent Executer**

In [ ]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

## **Invoke**

In [ ]:
agent_executor.invoke({"input":"what is attention mechanism"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'attention mechanism'}`


Page: Attention (machine learning)
Summary: Attention is a machine learning method that determines the relative importance of each component in a sequence relative to the other components in that sequence. In natural language processing, importance iAttention is a machine learning method that determines the relative importance of each component in a sequence relative to the other components in that sequence. In natural language processing, importance is assigned to different words in a sentence.  This allows a model to focus on the most relevant parts of the input when generating a response.  


> Finished chain.


{'input': 'what is attention mechanism',
 'output': 'Attention is a machine learning method that determines the relative importance of each component in a sequence relative to the other components in that sequence. In natural language processing, importance is assigned to different words in a sentence.  This allows a model to focus on the most relevant parts of the input when generating a response.  \n'}